In [1]:
import json
import pandas as pd
import plotly.express as px
import re

In [2]:
# Load the coordinates from the JSON file
with open("../../Extract/coordGPS/coordinates.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Convert the raw JSON data into a pandas DataFrame
# Each row contains the city name, latitude, and longitude as floats
df_coords = pd.DataFrame([
    {"city": city, "latitude": float(data["latitude"]), "longitude": float(data["longitude"])}
    for city, data in raw_data.items()
])

# Display the first 5 rows of the DataFrame to check the structure
df_coords.head()


city   latitude  longitude
0            Besançon, France  47.238022   6.024362
1  Bormes-les-Mimosas, France  43.150697   6.341928
2               Nimes, France  43.837425   4.360069
3               Dijon, France  47.321581   5.041470
4               Lille, France  50.636565   3.063528

In [3]:
# --- helpers ---
def normalize_city(name: str) -> str:
    """
    Minimal city name normalizer:
    - Removes trailing ", France"
    - Strips extra spaces
    - Converts to lowercase
    """
    name = re.sub(r",?\s*France$", "", name, flags=re.I)
    name = re.sub(r"\s+", " ", name).strip().lower()
    return name

# 1) clean label for display (without ", France")
df_coords["city_clean"] = df_coords["city"].str.replace(r",?\s*France$", "", regex=True)

# 2) define important cities (write them naturally; accents/tirets ok)
important_raw = {
    "Paris", "Lyon", "Marseille", "Toulouse", "Lille",
    "Strasbourg", "Grenoble", "Dijon", "Nîmes", "Avignon", 'La Rochelle', "Biarritz"
}
important_norm = {normalize_city(x) for x in important_raw}

# 3) label only if normalized city is in the important set
df_coords["label"] = df_coords["city"].apply(
    lambda s: df_coords.loc[df_coords["city"] == s, "city_clean"].iloc[0]
    if normalize_city(s) in important_norm else ""
)

# 4) build the map
fig = px.scatter_mapbox(
    df_coords,
    lat="latitude",
    lon="longitude",
    hover_name="city_clean",
    height=720,
    zoom=5.2
)

# 5) bigger bullets + force text display
fig.update_traces(
    mode="markers+text",                # <-- important to show text
    marker=dict(color="red", size=9),
    text=df_coords["label"],
    textposition="top center",
    textfont=dict(size=11)
)

# 6) layout
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_center={"lat": 46.6, "lon": 2.0},
    title="Carte des 35 villes touristiques françaises — Coordonnées GPS",
    title_x=0.5,
    margin={"r": 0, "t": 50, "l": 0, "b": 0}
)

fig.show()
